# Module: image_processing

The purpose of this notebook is to test the image_processing modules.
This will provide an example of how to use these modules in a pipeline.

In [8]:
from src import processing

import os
from glob import glob
from re import sub

## Extracting compressed images

In [2]:
#Directory containing compressed images
input_dir = 'data/human/registration/jacobians/absolute/smooth/'
input_dir = os.path.join(input_dir, '')

#List of paths to compressed images
input_files = glob(input_dir+'*.gz')
input_files[:5]

['data/human/registration/jacobians/absolute/smooth/d8_0148_01.extracted_fwhm_4vox.nii.gz',
 'data/human/registration/jacobians/absolute/smooth/41014_T1.extracted_fwhm_4vox.nii.gz',
 'data/human/registration/jacobians/absolute/smooth/d8_0393_01.extracted_fwhm_4vox.nii.gz',
 'data/human/registration/jacobians/absolute/smooth/21054_T1.extracted_fwhm_4vox.nii.gz',
 'data/human/registration/jacobians/absolute/smooth/d8_0204_01.extracted_fwhm_4vox.nii.gz']

In [3]:
# This seems to work as intended
# unzipped_files = processing.gunzip_files(infiles = input_files, 
#                                       keep = True,
#                                       parallel = True,
#                                       nproc = 8)

unzipped_files = glob(input_dir+'*.nii')
unzipped_files[:5]

['data/human/registration/jacobians/absolute/smooth/d8_0096_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/12013_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/21045_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/d8_0673_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/sub-1050792_ses-01_T1w.extracted_fwhm_4vox.nii']

***
## Converting image formats

The compressed files were in NIFTY format. We can keep these as is, or convert them to MINC format if we prefer. 

In [4]:
outdir = 'data/human/registration/jacobians/absolute/smooth_minc/'

# This seems to work as intended
# imgfiles = processing.convert_images(infiles = unzipped_files,
#                                   input_format = 'nifty',
#                                   output_format = 'minc',
#                                   outdir = imgdir,
#                                   keep = True,
#                                   parallel = True,
#                                   nproc = 8)
imgfiles = glob(outdir+'*.mnc')
imgfiles[:5]

['data/human/registration/jacobians/absolute/smooth_minc/sub-1050811_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/d8_0216_01.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050869_ses-01_run-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/31030_T1.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/d8_0713_02.extracted_fwhm_4vox.mnc']

*** 
## Computing effect sizes

In [35]:
imgdir = outdir
demographics = 'data/human/registration/DBM_input_demo_passedqc.csv'
maskfile = 'data/human/registration/reference_files/mask.mnc'
dataset = 1
ncontrols = 10
threshold = 5
es_dir = ('data/human/effect_sizes/absolute/'
          'resolution_{}_dataset_{}_ncontrols_{}_threshold_{}'
          .format(0.5, dataset, ncontrols, threshold))
es_dir = os.path.join(es_dir, '')
es_dir

'data/human/effect_sizes/absolute/resolution_0.5_dataset_1_ncontrols_10_threshold_5/'

In [36]:
# This seems to work as intended
# es_files = processing.calculate_human_effect_sizes(demographics = demographics,
#                                                    imgdir = imgdir,
#                                                    maskfile = maskfile,
#                                                    outdir = es_dir, 
#                                                    ncontrols = ncontrols,
#                                                    threshold = threshold,
#                                                    parallel = True,
#                                                    nproc = 4)
es_files = glob(es_dir+'*.mnc')
es_files = es_files[:10]
es_files

['data/human/effect_sizes/absolute/resolution_0.5_dataset_1_ncontrols_10_threshold_5/sub-1050158_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5.mnc',
 'data/human/effect_sizes/absolute/resolution_0.5_dataset_1_ncontrols_10_threshold_5/sub-1050100_ses-01_run-02_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5.mnc',
 'data/human/effect_sizes/absolute/resolution_0.5_dataset_1_ncontrols_10_threshold_5/sub-1050135_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5.mnc',
 'data/human/effect_sizes/absolute/resolution_0.5_dataset_1_ncontrols_10_threshold_5/sub-1050172_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5.mnc',
 'data/human/effect_sizes/absolute/resolution_0.5_dataset_1_ncontrols_10_threshold_5/sub-1050084_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5.mnc',
 'data/human/effect_sizes/absolute/resolution_0.5_dataset_1_ncontrols_10_threshold_5/sub-1050027_ses-01_run-03_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5.mnc',
 'data/human/effect_sizes/absolute/resolutio

---

## Resampling images

In [39]:
isostep = 3.0
es_dir_downsampled = sub(r'resolution_0.5', 
                         'resolution_{}'.format(isostep),
                         es_dir)
es_files_downsampled = processing.resample_images(infiles = es_files,
                                                  isostep = isostep,
                                                  outdir = es_dir_downsampled,
                                                  parallel = True,
                                                  nproc = 2)
es_files_downsampled

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.43s/it]


['data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050158_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
 'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050100_ses-01_run-02_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
 'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050135_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
 'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050172_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
 'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050084_ses-01_T1w.extracted_ES_res_0.5_data_1_nc_10_thresh_5_resampled_3.0.mnc',
 'data/human/effect_sizes/absolute/resolution_3.0_dataset_1_ncontrols_10_threshold_5/sub-1050027_ses-01_run-03_T1w.extracted_ES_res_0.5_da

We can also resampled individual images

In [40]:
infile = 'data/human/registration/reference_files/model.mnc'
isostep = 3.0
processing.resample_image(infile = infile,
                          isostep = isostep)

'data/human/registration/reference_files/model_resampled_3.0.mnc'

In [41]:
infile = 'data/human/registration/reference_files/mask.mnc'
processing.resample_image(infile = infile,
                          isostep = isostep)

'data/human/registration/reference_files/mask_resampled_3.0.mnc'

In [44]:
infiles = ['data/human/registration/reference_files/model.mnc',
           'data/human/registration/reference_files/mask.mnc']
isostep = 1.0
processing.resample_images(infiles = infiles,
                           isostep = isostep)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.58s/it]


['data/human/registration/reference_files/model_resampled_1.0.mnc',
 'data/human/registration/reference_files/mask_resampled_1.0.mnc']